In [1]:
from math import ceil, pi, e
from sympy import Symbol, diff, factorial, fraction, simplify, solve, symbols, Rational
from sympy.functions import sin, cos, tan
from termcolor import colored
from sympy.parsing.sympy_parser import parse_expr

In [2]:
def checkint(floatt):
  floatt = float(floatt)
  if floatt%1!=0:
    return floatt
  else:
    return int(floatt)

def truncate(floatt,n=4):
  floatt = str(float(floatt))
  indx = floatt.find('.')
  if indx!=-1:
    try:
      return checkint(floatt[:indx+n+1])
    except:
      return checkint(floatt)
  else:
    return checkint(floatt)

def printt(string,color="red",end='\n'):
  print(colored(string,color),end=end)

def printmtrx(mtrx,ts,symbs):
  printt('\nt',end='\t')
  [printt(i,end='\t') for i in ts]
  print()
  for i in range(len(symbs)):
    printt(symbs[i],end='\t')
    for j in range(len(ts)):
      print(mtrx[j][i],end='\t')
    print()

In [13]:
def makefunc(returnn,flag):
  if flag:
    def _func(values):
      return values.get(returnn)
  else:
    def _func(values):
      return parse_expr(returnn).subs(values)
  return _func

def printinitials(a,b,h,length,N):
  ts = [a]
  print(f'h = (b-a)/N =>\tN = ({b}-{a})/{h} = {N}')
  for i in range(N):
    ts.append(truncate(ts[-1]+h))
  [print(f't{i}={ts[i]}',end=' , ') for i in range(len(ts))]
  print()
  printt(f"{symbs[-1]}' = {fx}\n")

  us = [t]
  functions = list()

  for i in range(length):
    apos = "'"*i
    print(f'u{i+1}(t) = y{apos}(t)', end=' , ')
  print()
  for i in range(length):
    apos = "'"*i
    print(f'y{apos} = u{i+1}', end=' , ')
    us.append(f'u{i+1}')
  print()

  fy = str(fx)
  for i in range(length-1,-1,-1):
    apos = "'"*i
    fy = fy.replace(f'y{apos}',f'u{i+1}')
  print(f"u{length}' = {fy}\n")
  

  for i in range(1,length+1):
    if i!=length:
      print(f'f{i}{us} = {us[i+1]}')
      functions.append(makefunc(us[i+1],True))
    else:
      print(f'f{i}{us} = {fy}\n')
      functions.append(makefunc(fy,False))

  for i in range(1,length+1):
    print(f'u{i},0 = u{i}(t0) = u{i}({values[0]}) = {values[1][i-1]}')

  mtrx = list()
  for j in range(N+1):
    mtrx.append([0]*length)
  mtrx[0] = values[1]

  return ts, us, fy, mtrx, functions

def printmaineq(length):
  printt(f'\nui,j+1  = ui,j + h*fi(tj+h/2',end='',color='blue')
  for i in range(length):
    printt(f'; u{i+1},j+(h/2)*f{i+1}(tj',end='',color='blue')
    for j in range(length):
      printt(f', u{j+1},j',end='',color='blue')
    printt(')',end='',color='blue')
  print()

In [14]:
def secondorder(fx, symbs, values, interval, h):
  a,b = interval
  N = round((b-a)/h)
  length = len(symbs)
  
  ts, us, fy, mtrx, functions = printinitials(a,b,h,length, N)
  printmaineq(length)

  temp = truncate(h/2)
  for j in range(N):
    for i in range(1,length+1):
      printt(f'j={j}, i={i})')

      if i==1:
        print(f'u{i},{j+1} = {mtrx[j][i-1]} + {h}*f{i}({ts[j]}+{temp}',end='')
        for k in range(length):
          print(f', {mtrx[j][i-1]}+({temp})*f{k+1}({ts[j]}',end='')
          for l in range(length):
            print(f', {mtrx[j][l]}',end='')
          print(')',end='')
        print()

        stringg=f'({truncate(ts[j]+temp)}'
        print(f'     = {mtrx[j][i-1]} + {h}*f{i}',end='')
        for k in range(length):
          values = {'t':ts[j]}
          for l in range(length):
            values.update({us[l+1]:mtrx[j][l]})
          stringg+=f', {truncate(mtrx[j][k]+temp*functions[k](values))}'
        stringg+=')'
        print(f'{stringg}')

      else:
        print(f'u{i},{j+1} = {mtrx[j][i-1]} + {h}*f{i}{stringg}')
      tuplee = eval(stringg)
      values = {'t':tuplee[0]}
      for p in range(1,len(tuplee)):
        values.update({f'u{p}':tuplee[p]})
      mtrx[j+1][i-1] = truncate(mtrx[j][i-1]+truncate(h*functions[i-1](values)))
      print(f'     = {mtrx[j+1][i-1]}')
    printt('-------------------------------------------------------------','green')

  printt('Solution','blue')
  printmtrx(mtrx,ts,symbs)
  printt('-------------------------------------------------------------','green')
  return mtrx, ts

In [15]:
def comparefx(mtrx, realfx, symbs, t, ts):
  printt('\nTrue Values')
  realfxs = [realfx]
  for i in range(len(symbs)):
    apos = "'"*i
    if i!=0:
      realfxs.append(diff(realfxs[-1],t,1))
    print(f'y{apos}(t) = {realfxs[-1]}')
  print()
  
  for j in range(1,len(ts)):
    for i in range(len(symbs)):
      apos = "'"*i
      val = truncate(realfxs[i].subs(t,ts[j]))
      ans = truncate(abs(val - mtrx[j][i]))
      print(f'| y{apos}({ts[j]}) - u{i+1},{j+1} |  = | {val} - {mtrx[j][i]} | = {ans}')
    print()

In [16]:
y2, y1, y, t = symbols("y'' y' y t")      # set according to given fx
symbs = [y, y1, y2]                       # set according to given fx
fx = - 2*y2 + y1 + 2*y + e**t             # set according to given fx
values = [0,(1,2,0)]                      # eg y(0)=1, y'(0)=2, y''(0)=0
interval = (0,1)                          # a,b
h = 0.2                                   # h

mtrx, ts = secondorder(fx, symbs, values, interval, h)
realfx = (43/36)*e**t + (1/4)*e**(-t) - (4/9)*e**(-2*t) + (1/6)*t*e**t# set real equation if needed to compare
comparefx(mtrx, realfx, symbs, t, ts)

h = (b-a)/N =>	N = (1-0)/0.2 = 5
t0=0 , t1=0.2 , t2=0.4 , t3=0.6 , t4=0.8 , t5=1 , 
y''' = 2.71828182845905**t + 2*y + y' - 2*y''

u1(t) = y(t) , u2(t) = y'(t) , u3(t) = y''(t) , 
y = u1 , y' = u2 , y'' = u3 , 
u3' = 2.71828182845905**t + 2*u1 + u2 - 2*u3

f1[t, 'u1', 'u2', 'u3'] = u2
f2[t, 'u1', 'u2', 'u3'] = u3
f3[t, 'u1', 'u2', 'u3'] = 2.71828182845905**t + 2*u1 + u2 - 2*u3

u1,0 = u1(t0) = u1(0) = 1
u2,0 = u2(t0) = u2(0) = 2
u3,0 = u3(t0) = u3(0) = 0

ui,j+1  = ui,j + h*fi(tj+h/2; u1,j+(h/2)*f1(tj, u1,j, u2,j, u3,j); u2,j+(h/2)*f2(tj, u1,j, u2,j, u3,j); u3,j+(h/2)*f3(tj, u1,j, u2,j, u3,j)
j=0, i=1)
u1,1 = 1 + 0.2*f1(0+0.1, 1+(0.1)*f1(0, 1, 2, 0), 1+(0.1)*f2(0, 1, 2, 0), 1+(0.1)*f3(0, 1, 2, 0)
     = 1 + 0.2*f1(0.1, 1.2, 2, 0.5)
     = 1.4
j=0, i=2)
u2,1 = 2 + 0.2*f2(0.1, 1.2, 2, 0.5)
     = 2.1
j=0, i=3)
u3,1 = 0 + 0.2*f3(0.1, 1.2, 2, 0.5)
     = 0.901
-------------------------------------------------------------
j=1, i=1)
u1,2 = 1.4 + 0.2*f1(0.2+0.1, 1.4+(0.1)*f1(0.2, 1.4, 2.1, 0.

ESE

In [17]:
y1, y, t = symbols("y' y t")
symbs = [y, y1]
fx = -5*y - 4*y1
values = [0,(3,-5)]
interval = (0,0.5)
h = 0.5

mtrx, ts = secondorder(fx, symbs, values, interval, h)
#realfx = (43/36)*e**t + (1/4)*e**(-t) - (4/9)*e**(-2*t) + (1/6)*t*e**t# set real equation if needed to compare
#comparefx(mtrx, realfx, symbs, t, ts)

h = (b-a)/N =>	N = (0.5-0)/0.5 = 1
t0=0 , t1=0.5 , 
y'' = -5*y - 4*y'

u1(t) = y(t) , u2(t) = y'(t) , 
y = u1 , y' = u2 , 
u2' = -5*u1 - 4*u2

f1[t, 'u1', 'u2'] = u2
f2[t, 'u1', 'u2'] = -5*u1 - 4*u2

u1,0 = u1(t0) = u1(0) = 3
u2,0 = u2(t0) = u2(0) = -5

ui,j+1  = ui,j + h*fi(tj+h/2; u1,j+(h/2)*f1(tj, u1,j, u2,j); u2,j+(h/2)*f2(tj, u1,j, u2,j)
j=0, i=1)
u1,1 = 3 + 0.5*f1(0+0.25, 3+(0.25)*f1(0, 3, -5), 3+(0.25)*f2(0, 3, -5)
     = 3 + 0.5*f1(0.25, 1.75, -3.75)
     = 1.125
j=0, i=2)
u2,1 = -5 + 0.5*f2(0.25, 1.75, -3.75)
     = -1.875
-------------------------------------------------------------
Solution

t	0	0.5	
y	3	1.125	
y'	-5	-1.875	
-------------------------------------------------------------
